In [6]:
"""import self-written functions"""
from utility_functions import get_words,performance,compare_to_gpt_encoding
from BPE_function import bpe,get_best_merges

In [7]:
"""read training and test data"""
with open (r"corpora/Shakespeare_clean_train.txt", 'r') as f:
  text_train = f.read()
with open (r"corpora/Shakespeare_clean_valid.txt", 'r') as f:
  text_valid = f.read()
with open (r"corpora/Shakespeare_clean_test.txt", 'r') as f:
  text_test = f.read()


In [8]:
with open (r"corpora/Shakespeare_clean_valid.txt", 'r') as f:
  text_valid = f.read()
  
dict_train = get_words(text_train)
dict_valid = get_words(text_valid)
dict_test = get_words(text_test)

best_k,best_perplexity,best_n_gram,second_best_k,second_best_perplexity,second_best_n_gram,third_best_k,third_best_perplexity,third_best_n_gram, ks, n_gram_num, perplexities = get_best_merges(dict_train,text_train,dict_valid,text_valid,1000,100)
result = ["best k:",best_k,"best n gram", best_n_gram,"best k perplexity:",best_perplexity,"2nd best k:",second_best_k,"2nd best n gram", second_best_n_gram,"2nd best k perplexity:",second_best_perplexity,"3rd best k:",third_best_k,"3rd best n gram",third_best_n_gram,"3rd best k perplexity:",third_best_perplexity]
with open("k_merges_n_gram.txt", "w") as k_merges_n_gram:
  for item in result:
     k_merges_n_gram.write(f"{item}\n")

with open("k_merges_n_gram.txt", "r") as f:
    k_merges_n_gram = f.read().splitlines()

N-gram split for 1-gram, k = 100: [['the_'], ['t'], ['ra'], ['g'], ['e'], ['d'], ['y_'], ['of_'], ['an'], ['t']]
Old perplexity: 100000000.00106137
Perplexity: 95.83
N-gram split for 2-gram, k = 100: [['the_', 't'], ['t', 'ra'], ['ra', 'g'], ['g', 'e'], ['e', 'd'], ['d', 'y_'], ['y_', 'of_'], ['of_', 'an'], ['an', 't'], ['t', 'on']]
Old perplexity: 31.041010729441577
Perplexity: 30.48
N-gram split for 3-gram, k = 100: [['the_', 't', 'ra'], ['t', 'ra', 'g'], ['ra', 'g', 'e'], ['g', 'e', 'd'], ['e', 'd', 'y_'], ['d', 'y_', 'of_'], ['y_', 'of_', 'an'], ['of_', 'an', 't'], ['an', 't', 'on'], ['t', 'on', 'y_']]
Old perplexity: 53.91956844747087
Perplexity: 26.83
N-gram split for 4-gram, k = 100: [['the_', 't', 'ra', 'g'], ['t', 'ra', 'g', 'e'], ['ra', 'g', 'e', 'd'], ['g', 'e', 'd', 'y_'], ['e', 'd', 'y_', 'of_'], ['d', 'y_', 'of_', 'an'], ['y_', 'of_', 'an', 't'], ['of_', 'an', 't', 'on'], ['an', 't', 'on', 'y_'], ['t', 'on', 'y_', 'and_']]
Old perplexity: 1655.7131932966815
Perplexity: 79

In [9]:
"""train a dictionary of byte pairs and their respective counts on training and test data"""
#extract words from the text
dict_train = get_words(text_train)
dict_test = get_words(text_test)

best_k = int(k_merges_n_gram[1])
second_best_k = int(k_merges_n_gram[7])
third_best_k = int(k_merges_n_gram[13])


#builds the byte pair vocabulary and the sorted token frequency
vocab_train_best_k, sorted_token_freq_train_best_k, dict_matrix_train_best_k = bpe(dict_train,best_k)
vocab_train_2nd_best_k, sorted_token_freq_train_2nd_best_k, dict_matrix_train_2nd_best_k = bpe(dict_train,second_best_k)
vocab_train_3rd_best_k, sorted_token_freq_train_3rd_best_k, dict_matrix_train_3rd_best_k = bpe(dict_train,third_best_k)

In [10]:
"""test and print performance of training and test set"""
train_accuracy_best_k = performance(dict_train, vocab_train_best_k, 500)
train_accuracy_2nd_best_k = performance(dict_train, vocab_train_2nd_best_k, 500)
train_accuracy_3rd_best_k = performance(dict_train, vocab_train_3rd_best_k, 500)

test_accuracy_best_k = performance(dict_test, vocab_train_best_k, 500)
test_accuracy_2nd_best_k = performance(dict_test, vocab_train_2nd_best_k, 500)
test_accuracy_3rd_best_k = performance(dict_test, vocab_train_3rd_best_k, 500)

print("train accuracy best k: ", train_accuracy_best_k)
print("train accuracy 2nd best k: ", train_accuracy_2nd_best_k)
print("train accuracy 3rd best k: ", train_accuracy_3rd_best_k)

print("test accuracy best k: ", test_accuracy_best_k)
print("test accuracy 2nd best k: ", test_accuracy_2nd_best_k)
print("test accuracy 3rd best k: ", test_accuracy_3rd_best_k)

train accuracy best k:  7.3999999999999995
train accuracy 2nd best k:  7.3999999999999995
train accuracy 3rd best k:  13.200000000000001
test accuracy best k:  7.199999999999999
test accuracy 2nd best k:  7.199999999999999
test accuracy 3rd best k:  12.8


In [11]:
"""compare our BPE split to split of another system (in this case gpt 3.5)"""
#with open (file_path, 'r') as f:
#  text = f.read()
model_name="gpt-3.5-turbo"
tokens_train = compare_to_gpt_encoding(text_train, vocab_train_best_k, model_name)
tokens_train = compare_to_gpt_encoding(text_train, vocab_train_2nd_best_k, model_name)
tokens_train = compare_to_gpt_encoding(text_train, vocab_train_3rd_best_k, model_name)

tokens_test = compare_to_gpt_encoding(text_test, vocab_train_best_k, model_name)
tokens_test = compare_to_gpt_encoding(text_test, vocab_train_2nd_best_k, model_name)
tokens_test = compare_to_gpt_encoding(text_test, vocab_train_3rd_best_k, model_name)

0.8857142857142857
0.8857142857142857
0.8074866310160428
0.638095238095238
0.638095238095238
0.49732620320855614


In [12]:
"""run BPE split on different (non-shakespearean) test set and evaluate performance"""
with open ('corpora/friends.txt', 'r') as f:
  friends = f.read()

friends_dict = get_words(friends)
friends_accuracy = performance(friends_dict, vocab_train_best_k, 500)
friends_accuracy = performance(friends_dict, vocab_train_2nd_best_k, 500)
friends_accuracy = performance(friends_dict, vocab_train_3rd_best_k, 500)

print("Accuracy on Friends: ", friends_accuracy)

#also compare to gpt encoding of BPE split
tokens_friends = compare_to_gpt_encoding(friends, vocab_train_best_k, model_name)
tokens_friends = compare_to_gpt_encoding(friends, vocab_train_2nd_best_k, model_name)
tokens_friends = compare_to_gpt_encoding(friends, vocab_train_3rd_best_k, model_name)

FileNotFoundError: [Errno 2] No such file or directory: 'corpora/friends.txt'

In [ ]:
with open (r"corpora/Shakespeare_clean_valid.txt", 'r') as f:
  text_valid = f.read()
  
dict_train = get_words(text_train)
dict_valid = get_words(text_valid)
dict_test = get_words(text_test)

best_k,best_perplexity,best_n_gram,second_best_k,second_best_perplexity,second_best_n_gram,third_best_k,third_best_perplexity,third_best_n_gram = get_best_merges(dict_train,text_train,dict_valid,text_valid,1000,100)
result = ["best k:",best_k,"best n gram", best_n_gram,"best k perplexity:",best_perplexity,"2nd best k:",second_best_k,"2nd best n gram", second_best_n_gram,"2nd best k perplexity:",second_best_perplexity,"3rd best k:",third_best_k,"3rd best n gram",third_best_n_gram,"3rd best k perplexity:",third_best_perplexity]
with open("k_merges_n_gram.txt", "w") as k_merges_n_gram:
  for item in result:
     k_merges_n_gram.write(f"{item}\n")



N-gram split for 1-gram, k = 100: [['the_'], ['t'], ['ra'], ['g'], ['e'], ['d'], ['y_'], ['of_'], ['an'], ['t']]
Old perplexity: 100000000.00106137
Perplexity: 95.83
N-gram split for 2-gram, k = 100: [['the_', 't'], ['t', 'ra'], ['ra', 'g'], ['g', 'e'], ['e', 'd'], ['d', 'y_'], ['y_', 'of_'], ['of_', 'an'], ['an', 't'], ['t', 'on']]
Old perplexity: 31.041010729441577
Perplexity: 30.48
N-gram split for 3-gram, k = 100: [['the_', 't', 'ra'], ['t', 'ra', 'g'], ['ra', 'g', 'e'], ['g', 'e', 'd'], ['e', 'd', 'y_'], ['d', 'y_', 'of_'], ['y_', 'of_', 'an'], ['of_', 'an', 't'], ['an', 't', 'on'], ['t', 'on', 'y_']]
Old perplexity: 53.91956844747087
Perplexity: 26.83
N-gram split for 4-gram, k = 100: [['the_', 't', 'ra', 'g'], ['t', 'ra', 'g', 'e'], ['ra', 'g', 'e', 'd'], ['g', 'e', 'd', 'y_'], ['e', 'd', 'y_', 'of_'], ['d', 'y_', 'of_', 'an'], ['y_', 'of_', 'an', 't'], ['of_', 'an', 't', 'on'], ['an', 't', 'on', 'y_'], ['t', 'on', 'y_', 'and_']]
Old perplexity: 1655.7131932966815
Perplexity: 79